In [ ]:
try:
    from pyscf import gto, dft
    print("Found pyscf")
except:
    print("Can't import, installing via pip")
    !pip3 install pyscf
    from pyscf import gto

try:
    import numpy as np
    print("Found numpy")
except:
    print("Can't import, installing via pip")
    !pip3 install numpy
    import numpy as np

try:
    import matplotlib.pyplot as plt
    print("Found matplotlib")
except:
    print("Can't import, installing via pip")
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

try:
    from ase import Atoms
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write
    print("Found ase")
except:
    print("Can't import, installing via pip")
    !pip3 install ase
    from ase import Atoms    
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write

try:
    from pyscf.geomopt.geometric_solver import optimize
    print("Found geometry module")
except:
    print("Can't import, installing via pip")
    !pip3 install "pyscf[geomopt]"
    from pyscf.geomopt.geometric_solver import optimize

In [ ]:
def ase2pyscf():
    atomspyscf = []
    pos = atoms.get_positions()
    ele = atoms.get_chemical_symbols()
    numatoms = atoms.get_global_number_of_atoms()
    for atom in range(numatoms):
        elements = list(ele[atom])
        coords = np.ndarray.tolist(pos[atom])
        elencoord = elements + coords
        elencoordfix = ' '.join(map(str, elencoord))
        atomspyscf.append(elencoordfix)
    
    trueoutput = "; ".join(atomspyscf)
    
    return trueoutput

In [ ]:
moleculename = 'H2O'
atoms = molecule(moleculename)
inputatoms = ase2pyscf()

In [ ]:
mol = gto.Mole()
mol.build(atom = 'F -3.60733123  0.47284034 -0.00930631 ; C -0.91722962  0.43123757 -0.01265668 ; Cl 5.03944822  0.34185537 -0.01893118 ; H -0.21105407 -1.53952617 -0.03477183 ; H -0.16367267  1.38116357  1.69405262 ; H -0.16741984  1.41794644 -1.70001586',
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'angstrom')

#atom = inputatoms,
#atom = 'O 0.0 0.0 0.119262 ; H 0.0 0.763239 -0.477047 ; H 0.0 -0.763239 -0.477047',

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}

In [ ]:
dft_energies = []
def cb(envs):
  mf_hf = envs["g_scanner"].base
  dft_energies.append(mf_hf.e_tot)

In [ ]:
mol_eq = optimize(mf_hf, **conv_params, maxsteps=100, callback=cb)

In [ ]:
print(mol_eq.elements)
print(mol_eq.atom_coords())

In [ ]:
optmolecule = Atoms(mol_eq.elements, positions=mol_eq.atom_coords())

In [ ]:
view(optmolecule, viewer='x3d')

In [ ]:
print(dft_energies[-1])

In [ ]:
write(f'{round(dft_energies[-1] * 27.2114079527, 3)} -{moleculename}.xyz', optmolecule)

Find initial state or final state

In [ ]:
optimisedcoordinates = []

for element, coords in zip(mol_eq.elements, mol_eq.atom_coords()):
    optimisedcoordinates.append(f"{element} {' '.join(map(str, coords))}")

optimisedinput = ' ; '.join(optimisedcoordinates)


In [ ]:
mol = gto.Mole()
mol.build(atom = optimisedinput,
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'bohr')

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}
params = {"constraints": "constraints.txt"}

Optimise transition state close to scan value

In [ ]:
mol = gto.Mole()
mol.build(atom = """
F       -1.8204089865    0.2527953652   -0.0262120905
C       -0.2745895060    0.2225419907   -0.0194331119
Cl       2.0821379095    0.1821534941   -0.0026911966
H       -0.0481097714   -0.8499918220   -0.0295830951
H       -0.0191260542    0.7412624404    0.9104607373
H       -0.0128264004    0.7608567172   -0.9378247848""",
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'bohr')

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}
params = {'transition': True}

In [ ]:
mol_eq = optimize(mf_hf, **conv_params, **params, maxsteps=500, callback=cb)

Calculate thermoproperties

In [ ]:
from pyscf.hessian import thermo

In [ ]:
mol = gto.Mole()
mol.build(atom = """
F       -3.60733123       0.47284034      -0.00930631
C       -0.91722962       0.43123757      -0.01265668
Cl       5.03944822       0.34185537      -0.01893118
H       -0.21105407      -1.53952617      -0.03477183
H       -0.16367267       1.38116357       1.69405262
H       -0.16741984       1.41794644      -1.70001586""",
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'bohr')

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
hessian = mf_hf.Hessian().kernel()

In [ ]:
freq_info = thermo.harmonic_analysis(mf_hf.mol, hessian)
# Thermochemistry analysis at 298.15 K and 1 atmospheric pressure
thermo_info = thermo.thermo(mf_hf, freq_info['freq_au'], 298.15, 101325)

print('Total electronic energy')
print(thermo_info['E0' ])

print('Rotation constant')
print(thermo_info['rot_const'])

print('Zero-point energy')
print(thermo_info['ZPE'   ])

print('Internal energy at 0 K')
print(thermo_info['E_0K'  ])

print('Internal energy at 298.15 K')
print(thermo_info['E_tot' ])

print('Enthalpy energy at 298.15 K')
print(thermo_info['H_tot' ])

print('Gibbs free energy at 298.15 K')
print(thermo_info['G_tot' ])

print('Heat capacity at 298.15 K')
print(thermo_info['Cv_tot'])